# Module 2 – Session 3: Greedy Best-First Search

#### Exercise 1։ Trace Greedy Best-First Search (Conceptual)

- Start at **S (h=10)**
  - Neighbors added: A(6), B(7), E(1), **G(0)**
- GBFS picks the node with the smallest **h(n)** → **G(0)**
- **G** is the goal → reached 

##### Expanded nodes (in order)
**S → G**

##### Final path
**S → G**

##### Total cost
**2**

##### Optimal?
**Yes** — this is the shortest path available in the graph.

#### Exercise 2․ Designing Heuristics (Water Jug 5L–3L → get 2L in 3L)

**State representation:** `s = (x, y)`  
- `x` — water in 5L jug (0…5)  
- `y` — water in 3L jug (0…3)  
**Goal:** `y = 2`

##### Heuristic 1 — Volume gap in the 3L jug
**Definition:**  
\( h_1(s) = |\,y - 2\,| \)

**Why it’s good (cheap & informative):**  
- Measures how many liters the 3L jug is off from the target.
- Fast: O(1) to compute.
- Monotone intuition: the closer `y` is to 2, the smaller the value.

**Admissibility (intuition):**  
It never “overestimates” closeness in terms of liters difference to the goal in the 3L jug, so as a guidance signal for GBFS it is safe and consistent enough for ranking states.

##### Heuristic 2 — Volume gap + “penalty” for an unprepared 5L jug
**Definition:**  
\( h_2(s) = |\,y - 2\,| + \mathbf{1}[\,x \notin \{0,5\}\,] \)

Where \(\mathbf{1}[\cdot]\) is 1 if the condition is true, otherwise 0.

**Idea:**  
Often useful sequences begin with the 5L jug either **empty** or **full** (so pours are “maximally effective”).  
We keep the same volume gap as \(h_1\), but add a small penalty when the 5L jug is in an “intermediate” state (neither 0 nor 5), nudging the search to first **prepare** the 5L jug.

**Pros:**  
- Still O(1) to compute.  
- Gives a bit more structure to the search than \(h_1\), reducing detours through “messy” mid-states of the 5L jug.

**Notes:**  
- Both \(h_1\) and \(h_2\) are simple and cheap.  
- For Greedy Best-First Search, we only need a reasonable *ordering* by “closeness”, not exact step counts.


#### Exercise 3․ Implement Greedy Best-First Search (Coding)

Goal → Write and test a simple implementation of Greedy Best-First Search (GBFS).  
We’ll use a small sample graph and heuristic values.

In [1]:
# Example graph (simple road map)
graph = {
    'Yerevan': ['Gyumri', 'Vanadzor', 'Sevan'],
    'Gyumri': ['Vanadzor', 'Yerevan'],
    'Vanadzor': ['Dilizhan', 'Yerevan'],
    'Sevan': ['Dilizhan', 'Yerevan'],
    'Dilizhan': []
}

# Heuristic values (estimated straight-line distance to Dilizhan)
heuristics = {
    'Yerevan': 90,
    'Gyumri': 85,
    'Vanadzor': 35,
    'Sevan': 25,
    'Dilizhan': 0
}


In [11]:
from heapq import heappush, heappop

def greedy_bfs(graph, start, goal, heuristics):
    """Greedy Best-First Search using heuristic h(n) as priority."""
    visited = set()
    pq = []
    heappush(pq, (heuristics[start], [start]))

    while pq:
        priority, path = heappop(pq)
        node = path[-1]
        print(f"Visiting: {node} (h={heuristics[node]})")

        if node == goal:
            print("Goal reached")
            return path

        if node not in visited:
            visited.add(node)
            for neighbor in graph[node]:
                new_path = path + [neighbor]
                heappush(pq, (heuristics[neighbor], new_path))
    return None


In [12]:
path = greedy_bfs(graph, 'Yerevan', 'Dilizhan', heuristics)
print("\nFinal path found:", path)


Visiting: Yerevan (h=90)
Visiting: Sevan (h=25)
Visiting: Dilizhan (h=0)
Goal reached

Final path found: ['Yerevan', 'Sevan', 'Dilizhan']


In [13]:
# BFS and DFS for comparison
from collections import deque

def bfs(graph, start, goal):
    queue = deque([[start]])
    visited = set()
    while queue:
        path = queue.popleft()
        node = path[-1]
        if node == goal:
            return path
        if node not in visited:
            visited.add(node)
            for neighbor in graph[node]:
                queue.append(path + [neighbor])
    return None


def dfs(graph, start, goal):
    stack = [[start]]
    visited = set()
    while stack:
        path = stack.pop()
        node = path[-1]
        if node == goal:
            return path
        if node not in visited:
            visited.add(node)
            for neighbor in graph[node]:
                stack.append(path + [neighbor])
    return None


In [14]:
# ▶ Compare results
gbfs_path = greedy_bfs(graph, 'Yerevan', 'Dilizhan', heuristics)
bfs_path  = bfs(graph, 'Yerevan', 'Dilizhan')
dfs_path  = dfs(graph, 'Yerevan', 'Dilizhan')

print("\nGreedy Best-First Search:", gbfs_path)
print("Breadth-First Search:", bfs_path)
print("Depth-First Search:", dfs_path)


Visiting: Yerevan (h=90)
Visiting: Sevan (h=25)
Visiting: Dilizhan (h=0)
Goal reached

Greedy Best-First Search: ['Yerevan', 'Sevan', 'Dilizhan']
Breadth-First Search: ['Yerevan', 'Vanadzor', 'Dilizhan']
Depth-First Search: ['Yerevan', 'Sevan', 'Dilizhan']


In [15]:
# Inefficient / misleading heuristic
bad_heuristics = {
    'Yerevan': 10,
    'Gyumri': 1,       # misleads algorithm to go this way first
    'Vanadzor': 9,
    'Sevan': 8,
    'Dilizhan': 0
}

path_bad = greedy_bfs(graph, 'Yerevan', 'Dilizhan', bad_heuristics)
print("\nPath with inefficient heuristic:", path_bad)


Visiting: Yerevan (h=10)
Visiting: Gyumri (h=1)
Visiting: Sevan (h=8)
Visiting: Dilizhan (h=0)
Goal reached

Path with inefficient heuristic: ['Yerevan', 'Sevan', 'Dilizhan']


##### Observation

With the misleading heuristic, GBFS first chooses **Gyumri (h = 1)** although it is actually farther.  
The search becomes longer before reaching **Dilizhan**, proving that Greedy BFS does **not guarantee the shortest path** — it only relies on the heuristic estimate.
